In [190]:
import pandas as pd
import numpy as np
import math
from matplotlib import pyplot as plt
from fancyimpute import SoftImpute, BiScaler
from math import sqrt
from numpy import concatenate
from matplotlib import pyplot
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM


csv_data=read_csv('elephant.csv',header=0, index_col=0)
elephant_data=np.array(csv_data)  #column 3 and 4 are long and latitude
# latitude needs to be converted to x_y coordinates
#lat -> x, long->y

sampleSize=len(elephant_data)
r = 6371000 # meters, the earth radius
#function to turn lat and long to xy coordinates
def to_xy(coordinates, r):
    phi_0 = coordinates[:,1]
    cos_phi_0 = np.cos(np.radians(phi_0))
    lam = coordinates[:,0]
    phi = coordinates[:,1]
    x=(r * np.radians(lam) * cos_phi_0).reshape(len(coordinates),1)
    y=r * np.radians(phi).reshape(len(coordinates),1)
    xy=np.append(x,y,axis=1)
    return (xy )

#function to normalize the data
def normalize(matrix):
    normed=(matrix-matrix.mean(axis=0))/matrix.std(axis=0)
    return normed

# convert series to supervised learning
def series_to_supervised(data, n_in, n_out, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    # put it all together
    agg = concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan: # either drop NaN values or use imputation algorthms such as KNN imputation
        agg.dropna(inplace=True)
    return agg

'''
## KNN imputation

from fancyimpute import KNN
imputer = KNN(10) 
trainfillna = imputer.fit_transform(traindata)
from sklearn.preprocessing import OrdinalEncoder
encoder = OrdinalEncoder()
cat_cols = traindatacat.columns
def ordinalencode(train):
nonulls = np.array(data.dropna())
impute_reshape = nonulls.reshape(-1,1)
impute_ordinal = encoder.fit_transform(impute_reshape)
data.loc[data.notnull()] = np.squeeze(impute_ordinal)
return data
'''

def train_test_split(data,percent):
    n=len(data)
    nTrain=np.rint(n*percent)
    nTest=n-nTrain
    return int(nTrain), int(nTest)

def reform3D(data):
    data=np.array(data)
    data3d=data.reshape((data.shape[0],1,data.shape[1]))
    return data3d




In [225]:
elephant_lat=np.array(elephant_data[:,4]).astype(np.float64).reshape(sampleSize,1)
elephant_long=np.array(elephant_data[:,3]).astype(np.float64).reshape(sampleSize,1)

elephant_coordinates=np.append(elephant_lat,elephant_long,axis=1)
elephant_xy=to_xy(elephant_coordinates,r)

# normalize features
scaler = MinMaxScaler(feature_range=(0, 1))
scaled = scaler.fit_transform(elephant_xy)

sub=('00055877VTIC596')
index= csv_data["individual-local-identifier"].str.find(sub) 
index=index.values
elephant1=np.where(index!=-1)[0]
#elephant1=elephant1.reshape(1,len(elephant1))

elephant1_xy=elephant_xy[elephant1]
elephant1_xy.shape
reframed =np.array(series_to_supervised(elephant_xy, 60, 8))# the numbers are highly dependent on number of features
input_values=reframed[:,list(range(0,122))]
target_values=reframed[:,list(range(122,136))] # also highly dependent on features
#target_values=reframed.columns[list(range(122,136))] #the t + values are targets
#input_values=reframed.drop(reframed.columns[list(range(122,136))], axis=1, inplace=True).values
#print(reframed.head())


In [239]:
csv_data["timestamp"][0:7]

event-id
12751708    2003-11-25 11:36:13.000
12751709    2003-11-26 11:37:04.000
12751710    2003-11-28 10:59:03.000
12751711    2003-11-30 10:44:48.000
12751712    2003-12-01 11:39:18.000
12751713    2003-12-02 11:38:48.000
12751714    2003-12-04 11:38:59.000
Name: timestamp, dtype: object

In [204]:

nTrain, nTest=train_test_split(input_values,0.7)

input_values_train = input_values[:nTrain, :]
input_values_test = values[nTrain:, :]

target_values_train=target_values[:nTrain, :]
target_values_test=target_values[nTrain:,:]

#only 3d transform input data X

input_values_train3d=reform3D(input_values_train)
input_values_test3d=reform3D(input_values_test)


In [ ]:
regressor = Sequential()
regressor = Sequential()
regressor.add(Bidirectional(LSTM(units=200, return_sequences=True, input_shape = (x_train.shape[1],1) ) ))
regressor.add(Dropout(0.2))
regressor.add(LSTM(units= 200 , return_sequences=True))
regressor.add(Dropout(0.2))
regressor.add(LSTM(units= 200 , return_sequences=True))
regressor.add(Dropout(0.2))
regressor.add(LSTM(units= 200))
regressor.add(Dropout(0.2))
regressor.add(Dense(units = n_future,activation='linear'))
regressor.compile(optimizer='adam', loss='mean_squared_error',metrics=['acc'])
regressor.fit(x_train, y_train, epochs=500,batch_size=32 )

In [213]:
history = regressor.fit(input_values_train3d, target_values_train, epochs=50, batch_size=72, validation_data=(input_values_test3d, target_values_test), verbose=0)
 # plot history
pyplot.plot(history.history['loss'], label='train')
pyplot.plot(history.history['val_loss'], label='test')
pyplot.legend()
pyplot.show()
 
# make a prediction
yhat = model.predict(test_X)
test_X = test_X.reshape((test_X.shape[0], n_hours*n_features))
# invert scaling for forecast
inv_yhat = concatenate((yhat, test_X[:, -7:]), axis=1)
inv_yhat = scaler.inverse_transform(inv_yhat)
inv_yhat = inv_yhat[:,0]
# invert scaling for actual
test_y = test_y.reshape((len(test_y), 1))
inv_y = concatenate((test_y, test_X[:, -7:]), axis=1)
inv_y = scaler.inverse_transform(inv_y)
inv_y = inv_y[:,0]
# calculate RMSE
rmse = sqrt(mean_squared_error(inv_y, inv_yhat))
print('Test RMSE: %.3f' % rmse)   

ValueError: in user code:

    /Users/wenjinyuan/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:806 train_function  *
        return step_function(self, iterator)
    /Users/wenjinyuan/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:796 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /Users/wenjinyuan/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:1211 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /Users/wenjinyuan/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:2585 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /Users/wenjinyuan/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:2945 _call_for_each_replica
        return fn(*args, **kwargs)
    /Users/wenjinyuan/o

(23832, 1, 122) (10214, 1, 122) (23832, 14) (10214, 14)
